<a href="https://colab.research.google.com/github/Shruthiyadav75/Analysis-on-Investment-Opportunity/blob/main/DPDZERO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Objective: To build a simplified end-to-end data pipeline that mimics a real-world operational use case. My goal is to fetch, clean, transform, and report daily collection call data with basic validations and metrics.



Here we have three CSV files -
- Call Logs
- Agent Roster
- Disposition Summary

In [47]:
import pandas as pd
import numpy as np

In [48]:
call_logs = pd.read_csv("/content/call_logs.csv")

In [49]:
agent_roster = pd.read_csv("/content/agent_roster.csv")

In [50]:
disposition_summary = pd.read_csv("/content/disposition_summary.csv")

In [51]:
display(call_logs.head(5))
display(call_logs.shape)

,call_id,agent_id,org_id,installment_id,status,duration,created_ts,call_date
0,C5333,A020,O2,L1826,completed,5.68,2025-04-28T15:40:00,2025-04-28
1,C3045,A018,O1,L1996,no_answer,14.27,2025-04-28T02:41:00,2025-04-28
2,C5803,A018,O1,L1849,failed,11.01,2025-04-28T19:42:00,2025-04-28
3,C2139,A007,O1,L1046,connected,9.02,2025-04-28T09:52:00,2025-04-28
4,C4814,A003,O1,L1887,completed,2.42,2025-04-28T12:58:00,2025-04-28


(500, 8)

The call_logs dataframe contains 500 rows and 8 columns.

In [52]:
display(agent_roster.head(5))
display(agent_roster.shape)

,agent_id,users_first_name,users_last_name,users_office_location,org_id
0,A001,AgentFirst1,AgentLast1,Bangalore,O1
1,A002,AgentFirst2,AgentLast2,Delhi,O1
2,A003,AgentFirst3,AgentLast3,Mumbai,O1
3,A004,AgentFirst4,AgentLast4,Bangalore,O3
4,A005,AgentFirst5,AgentLast5,Bangalore,O3


(20, 5)

The agent_roster dataframe contains 20 rows and 5 columns.

In [53]:
display(disposition_summary.head(5))
display(disposition_summary.shape)

,agent_id,org_id,call_date,login_time
0,A001,O1,2025-04-28,11:58
1,A002,O1,2025-04-28,10:05
2,A003,O1,2025-04-28,10:24
3,A004,O3,2025-04-28,8:13
4,A005,O3,2025-04-28,NaN


(20, 4)

The disposition_summary dataframe contains 20 rows and 4 columns.

**To Find Duplicates**

call_logs data frame has 500 rows we can filter and display only the duplicate rows

In [54]:
duplicates = call_logs[call_logs.duplicated(keep=False)]
duplicates

,call_id,agent_id,org_id,installment_id,status,duration,created_ts,call_date


In [55]:
num_duplicates_call = call_logs.duplicated(keep=False).sum()
print(f"Number of duplicate rows in call_logs: {num_duplicates_call}")

Number of duplicate rows in call_logs: 0


In [56]:
num_duplicates_agent = agent_roster.duplicated(keep=False).sum()
print(f"Number of duplicate rows in agent_roster: {num_duplicates_agent}")

Number of duplicate rows in agent_roster: 0


In [57]:
num_duplicates_disposition = disposition_summary.duplicated(keep=False).sum()
print(f"Number of duplicate rows in disposition_summary: {num_duplicates_disposition}")

Number of duplicate rows in disposition_summary: 0


No duplicates found in all 3 data set

**Now lets check if there are any missing values in the entire DataFrame**

In [58]:
print(call_logs.isnull().any().any())

False


This returns False, it means there are no missing values in call_logs data frame

In [59]:
print(agent_roster.isnull().any().any())

False


This returns False, it means there are no missing values in agent_roster data frame

In [60]:
print(disposition_summary.isnull().any().any())

True


This returns True, it means there are missing values in disposition_summary data frame

In [61]:
missing_values = disposition_summary.isnull().sum()
print(missing_values)

agent_id      0
org_id        0
call_date     0
login_time    3
dtype: int64


In [62]:
disposition_summary = disposition_summary.dropna(subset=['login_time'])

In [63]:
missing_values = disposition_summary.isnull().sum()
print(missing_values)

agent_id      0
org_id        0
call_date     0
login_time    0
dtype: int64


In [64]:
call_logs.dtypes

,0
call_id,object
agent_id,object
org_id,object
installment_id,object
status,object
duration,float64
created_ts,object
call_date,object


Need to convert the data types of created_ts and call_date to date and time

In [65]:
call_logs['created_ts'] = pd.to_datetime(call_logs['created_ts'])
call_logs['call_date'] = pd.to_datetime(call_logs['call_date'])

In [66]:
call_logs.dtypes

,0
call_id,object
agent_id,object
org_id,object
installment_id,object
status,object
duration,float64
created_ts,datetime64[ns]
call_date,datetime64[ns]


In [67]:
agent_roster.dtypes

,0
agent_id,object
users_first_name,object
users_last_name,object
users_office_location,object
org_id,object


In [68]:
disposition_summary.dtypes

,0
agent_id,object
org_id,object
call_date,object
login_time,object


Need to convert the data types of call_date and login_time to date and time in the disposition_summary DataFrame.

In [69]:
disposition_summary['call_date'] = pd.to_datetime(disposition_summary['call_date'])
disposition_summary['login_time'] = pd.to_datetime(disposition_summary['login_time'])

<ipython-input-69-1ad025b46677>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  disposition_summary['login_time'] = pd.to_datetime(disposition_summary['login_time'])


In [70]:
disposition_summary.dtypes

,0
agent_id,object
org_id,object
call_date,datetime64[ns]
login_time,datetime64[ns]


Let us proceed with outlier detection in the call_logs DataFrame, as it contains 500 rows, which makes it appropriate for such analysis. However, for the other two DataFrames, which contain only 20 rows each, performing outlier detection may not be advisable due to the limited sample size

The IQR method is effective for medium-sized datasets like 500 rows.

Outlier detection is typically performed on numerical columns.

duration: This is a numerical column (float64), which represents a measurable quantity (the length of a call).

In [71]:
Q1 = call_logs['duration'].quantile(0.25)
Q3 = call_logs['duration'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = call_logs[(call_logs['duration'] < lower_bound) | (call_logs['duration'] > upper_bound)]
if outliers.empty:
    print("No outliers found in duration.")
else:
    print(f"Number of outlier rows in duration: {len(outliers)}")

No outliers found in duration.


**Let's merge the datasets using agent_id, org_id, and call_date while ensuring no data loss during the joins**

The outer join ensures no data loss that is all rows from both datasets are retained. Rows that don't have matches in the other dataset will have NaN values in the unmatched columns.

In [72]:
call_logs_agent_merged = pd.merge(
    call_logs,
    agent_roster,
    on=['agent_id', 'org_id'],
    how='outer',
    indicator='merge_logs_roster')
display(call_logs_agent_merged.head(5))
display(call_logs_agent_merged.shape)

,call_id,agent_id,org_id,installment_id,status,duration,created_ts,call_date,users_first_name,users_last_name,users_office_location,merge_logs_roster
0,C8508,A001,O1,L1636,completed,1.42,2025-04-28 02:30:00,2025-04-28,AgentFirst1,AgentLast1,Bangalore,both
1,C4164,A001,O1,L1408,failed,5.44,2025-04-28 09:30:00,2025-04-28,AgentFirst1,AgentLast1,Bangalore,both
2,C1757,A001,O1,L1250,no_answer,7.89,2025-04-28 20:51:00,2025-04-28,AgentFirst1,AgentLast1,Bangalore,both
3,C2746,A001,O1,L1797,no_answer,1.71,2025-04-28 07:28:00,2025-04-28,AgentFirst1,AgentLast1,Bangalore,both
4,C6763,A001,O1,L1305,no_answer,13.95,2025-04-28 14:18:00,2025-04-28,AgentFirst1,AgentLast1,Bangalore,both


(500, 12)

Next, merge the result with disposition_summary using agent_id, org_id, and call_date

In [73]:
final_merged_data = pd.merge(
    call_logs_agent_merged,
    disposition_summary,
    on=['agent_id', 'org_id', 'call_date'],
    how='outer',
    indicator='merge_final')
display(final_merged_data.head(5))
display(final_merged_data.shape)

,call_id,agent_id,org_id,installment_id,status,duration,created_ts,call_date,users_first_name,users_last_name,users_office_location,merge_logs_roster,login_time,merge_final
0,C8508,A001,O1,L1636,completed,1.42,2025-04-28 02:30:00,2025-04-28,AgentFirst1,AgentLast1,Bangalore,both,2025-05-05 11:58:00,both
1,C4164,A001,O1,L1408,failed,5.44,2025-04-28 09:30:00,2025-04-28,AgentFirst1,AgentLast1,Bangalore,both,2025-05-05 11:58:00,both
2,C1757,A001,O1,L1250,no_answer,7.89,2025-04-28 20:51:00,2025-04-28,AgentFirst1,AgentLast1,Bangalore,both,2025-05-05 11:58:00,both
3,C2746,A001,O1,L1797,no_answer,1.71,2025-04-28 07:28:00,2025-04-28,AgentFirst1,AgentLast1,Bangalore,both,2025-05-05 11:58:00,both
4,C6763,A001,O1,L1305,no_answer,13.95,2025-04-28 14:18:00,2025-04-28,AgentFirst1,AgentLast1,Bangalore,both,2025-05-05 11:58:00,both


(500, 14)

This approach retains all rows across all three datasets, ensuring no data loss during the joins.

**Let's handle mismatches appropriately to ensure data consistency and accuracy**

Checking for mismatches by using the indicator column to analyze unmatched rows during the join process.

Analyzing unmatched rows between call_logs and agent_roster using the indicator column to identify discrepancies

Lets find unexpected variations in the column values (e.g., extra spaces or case mismatches), normalize the column before filtering

In [74]:
call_logs_agent_merged['merge_logs_roster'] = call_logs_agent_merged['merge_logs_roster'].str.strip().str.lower()
unmatched_logs_roster = call_logs_agent_merged[call_logs_agent_merged['merge_logs_roster'] != 'both']
unmatched_logs_roster

,call_id,agent_id,org_id,installment_id,status,duration,created_ts,call_date,users_first_name,users_last_name,users_office_location,merge_logs_roster


In [75]:
print(call_logs_agent_merged['merge_logs_roster'].dtype)

object


In [76]:
non_matching = call_logs_agent_merged[~call_logs_agent_merged['merge_logs_roster'].isin(['both'])]
print(non_matching)

Empty DataFrame
Columns: [call_id, agent_id, org_id, installment_id, status, duration, created_ts, call_date, users_first_name, users_last_name, users_office_location, merge_logs_roster]
Index: []


In [77]:
print(call_logs_agent_merged[['merge_logs_roster']].value_counts())

merge_logs_roster
both                 500
Name: count, dtype: int64


In [78]:
print(call_logs_agent_merged['merge_logs_roster'].unique())

['both']


Rows with merge_final values of 'left_only' or 'right_only' highlight mismatches where records are present in one dataset but not in the other.

These mismatches can be investigated further to understand data discrepancies or missing values

In [79]:
final_merged_data['merge_final'] = final_merged_data['merge_final'].str.strip().str.lower()
unmatched_merge_final = final_merged_data[final_merged_data['merge_final'] != 'both']
unmatched_merge_final

,call_id,agent_id,org_id,installment_id,status,duration,created_ts,call_date,users_first_name,users_last_name,users_office_location,merge_logs_roster,login_time,merge_final
91,C3621,A005,O3,L1811,no_answer,4.63,2025-04-28 20:21:00,2025-04-28,AgentFirst5,AgentLast5,Bangalore,both,NaT,left_only
92,C5915,A005,O3,L1468,failed,13.84,2025-04-28 02:28:00,2025-04-28,AgentFirst5,AgentLast5,Bangalore,both,NaT,left_only
93,C5291,A005,O3,L1165,no_answer,2.20,2025-04-28 18:49:00,2025-04-28,AgentFirst5,AgentLast5,Bangalore,both,NaT,left_only
94,C9938,A005,O3,L1036,failed,1.21,2025-04-28 19:52:00,2025-04-28,AgentFirst5,AgentLast5,Bangalore,both,NaT,left_only
95,C4788,A005,O3,L1392,connected,3.51,2025-04-28 07:18:00,2025-04-28,AgentFirst5,AgentLast5,Bangalore,both,NaT,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,C4189,A018,O1,L1371,failed,10.12,2025-04-28 02:30:00,2025-04-28,AgentFirst18,AgentLast18,Bangalore,both,NaT,left_only
448,C5890,A018,O1,L1230,failed,10.62,2025-04-28 09:51:00,2025-04-28,AgentFirst18,AgentLast18,Bangalore,both,NaT,left_only
449,C5480,A018,O1,L1294,completed,1.36,2025-04-28 19:34:00,2025-04-28,AgentFirst18,AgentLast18,Bangalore,both,NaT,left_only
450,C7220,A018,O1,L1917,no_answer,14.80,2025-04-28 11:46:00,2025-04-28,AgentFirst18,AgentLast18,Bangalore,both,NaT,left_only


In [91]:
merge_summary = final_merged_data['merge_final'].value_counts()
merge_summary

,count
merge_final,
both,416
left_only,84


When handling mismatches after an outer join and seeing the following distribution:

416 records matched in both datasets (call_logs_agent_merged and disposition_summary).

84 records were only in call_logs_agent_merged, with no match in disposition_summary.


---


Plan for Missing Data

For the 84 unmatched records, missing values from disposition_summary will be filled as 'unknown' and thus prevents errors during analysis and clearly marks missing data for further review.

In [81]:
final_merged_data.loc[final_merged_data['merge_final'] == 'left_only', 'disposition_column'] = 'Unknown'

In [82]:
print(final_merged_data['merge_final'].value_counts())

merge_final
both         416
left_only     84
Name: count, dtype: int64


In [83]:
print(final_merged_data.isnull().sum())

call_id                    0
agent_id                   0
org_id                     0
installment_id             0
status                     0
duration                   0
created_ts                 0
call_date                  0
users_first_name           0
users_last_name            0
users_office_location      0
merge_logs_roster          0
login_time                84
merge_final                0
disposition_column       416
dtype: int64


Since login_time has 84 missing values, filling it with a placeholder

In [84]:
final_merged_data['login_time'] = final_merged_data['login_time'].fillna('Not Available')

In [85]:
print(final_merged_data.isnull().sum())

call_id                    0
agent_id                   0
org_id                     0
installment_id             0
status                     0
duration                   0
created_ts                 0
call_date                  0
users_first_name           0
users_last_name            0
users_office_location      0
merge_logs_roster          0
login_time                 0
merge_final                0
disposition_column       416
dtype: int64


In [86]:
def count_completed_calls(status_series):
    return (status_series == 'completed').sum()

def avg_call_duration(duration_series):
    return duration_series.mean() / 60 if not duration_series.empty else 0

def compute_presence(login_time_series):
    return int(login_time_series.notnull().any())

metrics = final_merged_data.groupby(['agent_id', 'call_date']).agg(
    Total_Calls_Made=('call_id', 'count'),
    Unique_Loans_Contacted=('installment_id', 'nunique'),
    Completed_Calls=('status', count_completed_calls),
    Avg_Call_Duration=('duration', avg_call_duration),
    Presence=('login_time', compute_presence)
).reset_index()

metrics['Connect_Rate'] = metrics['Completed_Calls'] / metrics['Total_Calls_Made']
display(metrics)

,agent_id,call_date,Total_Calls_Made,Unique_Loans_Contacted,Completed_Calls,Avg_Call_Duration,Presence,Connect_Rate
0,A001,2025-04-28,20,20,2,0.114050,1,0.100000
1,A002,2025-04-28,23,23,3,0.130464,1,0.130435
2,A003,2025-04-28,21,21,8,0.119492,1,0.380952
3,A004,2025-04-28,27,27,4,0.134278,1,0.148148
4,A005,2025-04-28,29,28,4,0.117655,1,0.137931
5,A006,2025-04-28,16,16,2,0.147292,1,0.125000
6,A007,2025-04-28,24,23,5,0.117340,1,0.208333
7,A008,2025-04-28,29,29,8,0.110052,1,0.275862
8,A009,2025-04-28,26,25,7,0.123122,1,0.269231
9,A010,2025-04-28,38,38,7,0.134649,1,0.184211


In [92]:
metrics.to_csv('agent_performance_summary.csv', index=False)

top_performer = metrics.loc[metrics['Connect_Rate'].idxmax()]
top_performer_name = final_merged_data.loc[
    final_merged_data['agent_id'] == top_performer['agent_id'], 'users_first_name'
].iloc[0]
total_active_agents = metrics.shape[0]
average_duration = metrics['Avg_Call_Duration'].mean()

slack_message = f"""
Agent Summary for {metrics['call_date'].max()}
Top Performer: {top_performer_name} ({top_performer['Connect_Rate']*100:.2f}% connect rate)
Total Active Agents: {total_active_agents}
Average Duration: {average_duration:.1f} min
"""
print(slack_message)


Agent Summary for 2025-04-28 00:00:00  
Top Performer: AgentFirst3 (38.10% connect rate)  
Total Active Agents: 20  
Average Duration: 0.1 min

